In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import os
from PIL import Image
from segmentation_tools.visualisation import draw_sem_seg, VisImage
from segmentation_tools.label_data import get_ade150, get_nyu40, get_scannet_all, get_wordnet, get_replica
from segmentation_tools.label_mappings import set_ids_according_to_names, \
        match_scannet_ade150, \
        match_scannet_nyu40, \
        match_ade150_nyu40, \
        match_scannet_wordnet199, \
        LabelMatcher, \
        set_colors
from pathlib import Path
from scripts import segmentation_eval
from scripts import pointcloud_eval
from scripts import segmentation_consensus
import skimage

key = 50
scene_dir = Path('/home/weders/scratch/scratch/scannetter/room_0_1')
img = cv2.imread(f'{scene_dir}/rgb/rgb_{key}.png')[..., ::-1]
label = cv2.imread(f'{scene_dir}/semantic_class/semantic_class_{key}.png',
                   cv2.IMREAD_UNCHANGED)
# our_label = cv2.imread(f'{scene_dir}/label_agile3d/{key}.png', cv2.IMREAD_UNCHANGED)

consensus = cv2.imread(f'{scene_dir}/pred_wn_consensus/{key}.png',
                   cv2.IMREAD_UNCHANGED)
# sdfstudio = cv2.imread(f'{scene_dir}/pred_sdfstudio_2023-07-19_164624/{(key // 2):05d}.png',
#                    cv2.IMREAD_UNCHANGED)



classid2wn = {x['id']: x['name'] for x in get_wordnet(label_key='wn199-merged-v2')}
scannet_id_to_name = {x['id'] : x['name'] for x in get_scannet_all()}
scannet_id_to_color = {x['id'] : x['color'] for x in get_scannet_all()}

print(classid2wn)

keys = sorted(
            int(x.name.split('.')[0].split('_')[-1])
            for x in (scene_dir / 'rgb').iterdir())
label_template = 'semantic_class/semantic_class_{k}.png'
label_space = 'id'
plt.figure(figsize=(20, 10))
plt.imshow(img)
plt.gca().axis('off')
plt.show()
_, plots = plt.subplots(1, 4, figsize=(30, 10))
vis = VisImage(img)
# draw_sem_seg(our_label, vis, 
#             classes=[x['name'] for x in sorted(get_wordnet(), key=lambda x: x['id'])],
#             colors=[x['color'] for x in sorted(get_wordnet(), key=lambda x: x['id'])])
# plots[0].imshow(vis.get_image())
plots[0].axis('off')
plots[0].set_title('Ground Truth')
vis = VisImage(img)
draw_sem_seg(label, vis, 
             classes=[scannet_id_to_name[i] if i in scannet_id_to_name else 'unknown' for i in range(2000)],
             colors=[scannet_id_to_color[i] if i in scannet_id_to_name else [0, 0, 0] for i in range(2000)],)
plots[1].imshow(vis.get_image())
plots[1].axis('off')
plots[1].set_title('ScanNet')
# vis = VisImage(img)
# draw_sem_seg(sdfstudio, vis, 
#             classes=[x['name'] for x in sorted(get_wordnet(), key=lambda x: x['id'])],
#             colors=[x['color'] for x in sorted(get_wordnet(), key=lambda x: x['id'])])
#plots[2].imshow(vis.get_image())
# plots[2].axis('off')
plots[2].set_title('LabelMaker3D')
vis = VisImage(img)
draw_sem_seg(consensus, vis, 
             classes=[x['name'] for x in sorted(get_wordnet(), key=lambda x: x['id'])],
             colors=[x['color'] for x in sorted(get_wordnet(), key=lambda x: x['id'])])
plots[3].imshow(vis.get_image())
plots[3].axis('off')
plots[3].set_title('Consensus')
plt.show()

In [ ]:
# iterate over all frames for scene

scene = 'scene0000_00'
scene_dir = Path(f'/home/weders/scratch/scratch/scannetter/{scene}')
sdfstudio_pred = 'pred_sdfstudio_2023-07-30_112430'

keys = sorted([int(k.split('/')[-1].replace('.jpg', '')) for k in os.listdir((scene_dir / 'color'))])

for idx, key in enumerate(keys):
    
    if idx % 20 != 0:
        continue
        
    img = cv2.imread(f'{scene_dir}/color/{key}.jpg')[..., ::-1]
    label = cv2.imread(f'{scene_dir}/label-filt/{key}.png',
                       cv2.IMREAD_UNCHANGED)
    our_label = cv2.imread(f'{scene_dir}/label_agile3d/{key}.png', cv2.IMREAD_UNCHANGED)

    consensus = cv2.imread(f'{scene_dir}/pred_consensus_5_scannet/{key}.png',
                       cv2.IMREAD_UNCHANGED)
    sdfstudio = cv2.imread(f'{scene_dir}/{sdfstudio_pred}/{(key // 2):05d}.png',
                       cv2.IMREAD_UNCHANGED)

    classid2wn = {x['id']: x['name'] for x in get_wordnet()}
    scannet_id_to_name = {x['id'] : x['name'] for x in get_scannet_all()}
    scannet_id_to_color = {x['id'] : x['color'] for x in get_scannet_all()}

    classes = [x['name'] for x in sorted(get_wordnet(label_key='wn199-merged-v2'), key=lambda x: x['id'])]
    
    keys = sorted(
                int(x.name.split('.')[0])
                for x in (scene_dir / 'color').iterdir())
    label_template = 'label-filt/{k}.png'
    label_space = 'id'
#     plt.figure(figsize=(20, 10))
#     plt.imshow(img)
#     plt.gca().axis('off')
#     plt.show()
    _, plots = plt.subplots(1, 5, figsize=(40, 10))
    
    plots[0].imshow(img)
    plots[0].axis('off')
    plots[0].set_title('Image')
    
    vis = VisImage(img)
    draw_sem_seg(our_label, vis, 
                 classes={x['id']: x['name'] for x in sorted(get_wordnet(label_key='wn199-merged-v2'), key=lambda x: x['id'])},
                 colors=[x['color'] for x in sorted(get_wordnet(label_key='wn199-merged-v2'), key=lambda x: x['id'])])
    plots[1].imshow(vis.get_image())
    plots[1].axis('off')
    plots[1].set_title('Ground Truth')
    vis = VisImage(img)
    draw_sem_seg(label, vis, 
                 classes=[scannet_id_to_name[i] if i in scannet_id_to_name else 'unknown' for i in range(2000)],
                 colors=[scannet_id_to_color[i] if i in scannet_id_to_name else [0, 0, 0] for i in range(2000)],)
    plots[2].imshow(vis.get_image())
    plots[2].axis('off')
    plots[2].set_title('ScanNet')
    vis = VisImage(img)
    draw_sem_seg(sdfstudio, vis, 
                 classes=[x['name'] for x in sorted(get_wordnet(label_key='wn199-merged-v2'), key=lambda x: x['id'])],
                 colors=[x['color'] for x in sorted(get_wordnet(label_key='wn199-merged-v2'), key=lambda x: x['id'])])
    plots[3].imshow(vis.get_image())
    plots[3].axis('off')
    plots[3].set_title('LabelMaker3D')
    vis = VisImage(img)
    draw_sem_seg(consensus, vis, 
                 classes=[x['name'] for x in sorted(get_wordnet(label_key='wn199-merged-v2'), key=lambda x: x['id'])],
                 colors=[x['color'] for x in sorted(get_wordnet(label_key='wn199-merged-v2'), key=lambda x: x['id'])])
    plots[4].imshow(vis.get_image())
    plots[4].axis('off')
    plots[4].set_title('Consensus')
    plt.tight_layout()
    plt.show()
    

In [ ]:
set_colors()

In [ ]:
vis = VisImage(img)
draw_sem_seg(our_label, vis, classes=[x['name'] for x in sorted(get_wordnet(), key=lambda x: x['id'])])
plt.figure(figsize=(20, 10))
plt.imshow(vis.get_image())
plt.gca().axis('off')
plt.show()

In [ ]:
vis = VisImage(cv2.resize(img, (1296, 968)))
scannet_id_to_name = {x['id'] : x['name'] for x in get_scannet_all()}
draw_sem_seg(label, vis, classes=[scannet_id_to_name[i] if i in scannet_id_to_name else 'unknown' for i in range(2000)])
plt.figure(figsize=(20, 10))
plt.imshow(vis.get_image())
plt.gca().axis('off')
plt.show()

In [ ]:
matcher = LabelMatcher('id', 'wn199', verbose=False)
vis = VisImage(img, scale=1)
draw_sem_seg(matcher.match(label, our_label), vis, classes=['mismatch', 'match', 'unknown'])
plt.figure(figsize=(20, 10))
plt.imshow(vis.get_image())
plt.gca().axis('off')
plt.show()

In [ ]:
inst_sam = cv2.imread(f'{scene_dir}/pred_sam/{key}.png',
                   cv2.IMREAD_UNCHANGED)
vis = VisImage(img, scale=1)
draw_sem_seg(inst_sam, vis, classes=[str(i) for i in range(200)])
plt.figure(figsize=(20, 10))
plt.imshow(vis.get_image())
plt.gca().axis('off')

plt.show()

In [ ]:
ovseg_wn = cv2.imread(f'{scene_dir}/pred_ovseg_wn_nodef/{key}.png',
                   cv2.IMREAD_UNCHANGED)
vis = VisImage(img, scale=1)
draw_sem_seg(ovseg_wn, vis, 
             classes=[x['name'] for x in sorted(get_wordnet(), key=lambda x: x['id'])],
             colors=[x['color'] for x in sorted(get_wordnet(), key=lambda x: x['id'])])
plt.figure(figsize=(20, 10))
plt.imshow(vis.get_image())
plt.gca().axis('off')
plt.show()

In [ ]:
intern = cv2.imread(f'{scene_dir}/pred_internimage/{key}.png',
                   cv2.IMREAD_UNCHANGED)
vis = VisImage(img, scale=1)
draw_sem_seg(intern, vis, classes=[x['name'] for x in sorted(get_ade150(), key=lambda x: x['id'])])
plt.figure(figsize=(20, 10))
plt.imshow(vis.get_image())
plt.gca().axis('off')

plt.show()

In [ ]:
cmx = cv2.imread(f'{scene_dir}/pred_cmx/{key}.png',
                   cv2.IMREAD_UNCHANGED)
vis = VisImage(cv2.resize(img, (640, 480)))
draw_sem_seg(cmx, vis, classes=['unknown'] + [x['name'] for x in sorted(get_nyu40(), key=lambda x: int(x['id']))])
#draw_sem_seg(cmx, vis, classes=[str(x) for x in range(50)])
plt.figure(figsize=(20, 10))
plt.imshow(vis.get_image())
plt.gca().axis('off')
plt.show()

In [ ]:
from scripts.segmentation_consensus import PredictorVoting

votes = PredictorVoting()
n_votes, pred_vote = votes.voting(
    nyu40_predictions=[cv2.resize(cmx, (1296, 968), interpolation=cv2.INTER_NEAREST)],
    ade20k_predictions=[intern],
    wn199_predictions=[ovseg_wn]
)
vis = VisImage(img)
draw_sem_seg(pred_vote, vis, classes=[x['name'] for x in sorted(get_wordnet(), key=lambda x: x['id'])])
plt.figure(figsize=(20, 10))
plt.imshow(vis.get_image())
plt.gca().axis('off')
plt.show()
vis = VisImage(img)
draw_sem_seg(n_votes, vis, classes=[str(x) for x in range(10)])
plt.figure(figsize=(20, 10))
plt.imshow(vis.get_image())
plt.gca().axis('off')
plt.show()

In [ ]:
from segmentation_tools.scannet_200_labels import CLASS_LABELS_200
scannet_id_to_name = {x['id']: x['name'] for x in get_scannet_all()}
names = []
for x in range(1500):
    if x in scannet_id_to_name:
        names.append(scannet_id_to_name[x])
    else:
        names.append('unknown')
mask3d = cv2.imread(f'{scene_dir}/pred_mask3d_rendered/{key}.png',
                   cv2.IMREAD_UNCHANGED)
vis = VisImage(img)
draw_sem_seg(mask3d, vis, classes=names, alpha=0.8)
plt.figure(figsize=(20, 10))
plt.imshow(vis.get_image())
plt.gca().axis('off')
plt.show()

In [ ]:
consensus = cv2.imread(f'{scene_dir}/pred_consensus_5_scannet/{key}.png',
                   cv2.IMREAD_UNCHANGED)
vis = VisImage(img)
draw_sem_seg(consensus, vis, 
             classes=[x['name'] for x in sorted(get_wordnet(), key=lambda x: x['id'])],
             colors=[x['color'] for x in sorted(get_wordnet(), key=lambda x: x['id'])])
plt.figure(figsize=(20, 10))
plt.imshow(vis.get_image())
plt.gca().axis('off')
plt.show()

In [ ]:
consensus = cv2.imread(f'{scene_dir}/pred_consensus_noscannet/{key}.png',
                   cv2.IMREAD_UNCHANGED)
vis = VisImage(img)
draw_sem_seg(consensus, vis, 
             classes=[x['name'] for x in sorted(get_wordnet(), key=lambda x: x['id'])],
             colors=[x['color'] for x in sorted(get_wordnet(), key=lambda x: x['id'])])
plt.figure(figsize=(20, 10))
plt.imshow(vis.get_image())
plt.gca().axis('off')
plt.show()

In [ ]:
sdfstudio = cv2.imread(f'{scene_dir}/pred_sdfstudio_2023-07-19_164624/{(key // 2):05d}.png',
                   cv2.IMREAD_UNCHANGED)
vis = VisImage(img)
draw_sem_seg(sdfstudio, vis, 
             classes=[x['name'] for x in sorted(get_wordnet(), key=lambda x: x['id'])],
             colors=[x['color'] for x in sorted(get_wordnet(), key=lambda x: x['id'])])
plt.figure(figsize=(20, 10))
plt.imshow(vis.get_image())
plt.gca().axis('off')
plt.show()

# 2d Eval

In [ ]:
PATH_PREFIX = '/home/weders/'

scenes = [
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0000_00/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0458_00/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0164_02/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0518_00/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0474_01/',
]

# scannet voting weight 5 patchloss 0.01
sdfstudio_paths = [
                   'pred_sdfstudio_2023-07-28_195241/{k:05d}.png', #000
                   'pred_sdfstudio_2023-07-28_154310/{k:05d}.png', #458
                   'pred_sdfstudio_2023-07-28_154200/{k:05d}.png', #164
                   'pred_sdfstudio_2023-07-28_154206/{k:05d}.png', #518
                   'pred_sdfstudio_2023-07-28_154310/{k:05d}.png',#474
                  ]

# scannet voting weight 5
sdfstudio_paths = [
                   'pred_sdfstudio_2023-07-28_171544/{k:05d}.png', #000
                   'pred_sdfstudio_2023-07-28_154244/{k:05d}.png', #458
                   'pred_sdfstudio_2023-07-28_154207/{k:05d}.png', #164
                   'pred_sdfstudio_2023-07-28_154200/{k:05d}.png', #518
                   'pred_sdfstudio_2023-07-28_154249/{k:05d}.png',#474
                  ]


# labelmaker3d final
sdfstudio_paths = [
                   'pred_sdfstudio_2023-07-30_112430/{k:05d}.png', #000
                   'pred_sdfstudio_2023-07-30_104953/{k:05d}.png', #458
                   'pred_sdfstudio_2023-07-30_104700/{k:05d}.png', #164
                   'pred_sdfstudio_2023-07-30_104735/{k:05d}.png', #518
                   'pred_sdfstudio_2023-07-30_105014/{k:05d}.png',#474
                  ]

# semantic nerf predictions
semantic_nerf_paths = [
                   'pred_sdfstudio_2023-08-02_003056/{k:05d}.png', #000
                   'pred_sdfstudio_2023-08-02_000600/{k:05d}.png', #458
                   'pred_sdfstudio_2023-08-02_000339/{k:05d}.png', #164
                   'pred_sdfstudio_2023-08-02_000401/{k:05d}.png', #518
                   'pred_sdfstudio_2023-08-02_000611/{k:05d}.png',#474
                  ]

# semantic nerf predictions
noscannet_paths = [
                   'pred_sdfstudio_2023-08-02_015512/{k:05d}.png', #000
                   'pred_sdfstudio_2023-08-02_002220/{k:05d}.png', #458
                   'pred_sdfstudio_2023-08-02_002042/{k:05d}.png', #164
                   'pred_sdfstudio_2023-08-02_083237/{k:05d}.png', #518
                   'pred_sdfstudio_2023-08-02_083537/{k:05d}.png',#474
                  ]

In [ ]:
scene = 'scene0458_00'
scene_idx = -4

scenes_ = [scenes[scene_idx]]
overwrite_confmat = True
pd.set_option("display.float_format", "{:.3f}".format)

matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'id',
                                'wn199-merged-v2',
                                pred_template='label-filt/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)

d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.DataFrame(d, index=['scannet'])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199',
                                'wn199-merged-v2',
                                subsampling=2,
                                pred_template=sdfstudio_paths[scene_idx],
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['labelmaker3d'])])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                pred_template='pred_consensus_5_scannet/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['consensus'])])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                pred_template='pred_consensus_noscannet/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['cons. w\o sn'])])




columns = ['tAcc', 'mIoU', 'mAcc']
for i, x in enumerate(matcher.right_ids):
    if confmat.sum(0)[i+1] > 0:
        columns.append(classid2wn[x])
print(scene)
df[columns].sort_values('mIoU', ascending=False).T


In [ ]:
scene = 'scene0000_00'
scene_idx = -5

scenes_ = [scenes[scene_idx]]
overwrite_confmat = True
pd.set_option("display.float_format", "{:.3f}".format)

matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'id',
                                'wn199-merged-v2',
                                pred_template='label-filt/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)

d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.DataFrame(d, index=['scannet'])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199',
                                'wn199-merged-v2',
                                subsampling=2,
                                pred_template=sdfstudio_paths[scene_idx],
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['labelmaker3d'])])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                pred_template='pred_consensus_5_scannet/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['consensus'])])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                pred_template='pred_consensus_noscannet/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['cons. w\o sn'])])


columns = ['tAcc', 'mIoU', 'mAcc']
for i, x in enumerate(matcher.right_ids):
    if confmat.sum(0)[i+1] > 0:
        columns.append(classid2wn[x])
print(scene)
df[columns].sort_values('mIoU', ascending=False).T


In [ ]:
# scene0164_02

scene = 'scene0164_02'
scene_idx = -3

scenes_ = [scenes[scene_idx]]
overwrite_confmat = True
pd.set_option("display.float_format", "{:.3f}".format)

matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'id',
                                'wn199-merged-v2',
                                pred_template='label-filt/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)

d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.DataFrame(d, index=['scannet'])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199',
                                'wn199-merged-v2',
                                subsampling=2,
                                pred_template=sdfstudio_paths[scene_idx],
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['labelmaker3d'])])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                pred_template='pred_consensus_5_scannet/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['consensus'])])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                pred_template='pred_consensus_noscannet/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['cons. w\o sn'])])


columns = ['tAcc', 'mIoU', 'mAcc']
for i, x in enumerate(matcher.right_ids):
    if confmat.sum(0)[i+1] > 0:
        columns.append(classid2wn[x])
print(scene)
df[columns].sort_values('mIoU', ascending=False).T


In [ ]:
scene = 'scene0518_00'
scene_idx = -2

scenes_ = [scenes[scene_idx]]
overwrite_confmat = True
pd.set_option("display.float_format", "{:.3f}".format)

matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'id',
                                'wn199-merged-v2',
                                pred_template='label-filt/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)

d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.DataFrame(d, index=['scannet'])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199',
                                'wn199-merged-v2',
                                subsampling=2,
                                pred_template=sdfstudio_paths[scene_idx],
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['labelmaker3d'])])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                pred_template='pred_consensus_5_scannet/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['consensus'])])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                pred_template='pred_consensus_noscannet/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['cons. w\o sn'])])


columns = ['tAcc', 'mIoU', 'mAcc']
for i, x in enumerate(matcher.right_ids):
    if confmat.sum(0)[i+1] > 0:
        columns.append(classid2wn[x])
print(scene)
df[columns].sort_values('mIoU', ascending=False).T


In [ ]:
scene = 'scene0474_01'
scene_idx = -1

scenes_ = [scenes[scene_idx]]
overwrite_confmat = True
pd.set_option("display.float_format", "{:.3f}".format)

matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'id',
                                'wn199-merged-v2',
                                subsampling=1,
                                pred_template='label-filt/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)

d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.DataFrame(d, index=['scannet'])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199',
                                'wn199-merged-v2',
                                subsampling=2,
                                pred_template=sdfstudio_paths[scene_idx],
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['labelmaker3d'])])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                subsampling=1,
                                pred_template='pred_consensus_5_scannet/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['consensus'])])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes_,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                subsampling=1,
                                pred_template='pred_consensus_noscannet/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['cons. w\o sn'])])


columns = ['tAcc', 'mIoU', 'mAcc']
for i, x in enumerate(matcher.right_ids):
    if confmat.sum(0)[i+1] > 0:
        columns.append(classid2wn[x])
print(scene)
df[columns].sort_values('mIoU', ascending=False).T


In [ ]:


overwrite_confmat = True
pd.set_option("display.float_format", "{:.3f}".format)
pd.set_option('display.max_rows', 500)

print('ScanNet')
matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'id',
                                'wn199-merged-v2',
                                pred_template='label-filt/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                subsampling=1,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.DataFrame(d, index=['scannet']);

print('LabelMaker')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'wn199',
                                'wn199-merged-v2',
                                subsampling=2,
                                pred_template=sdfstudio_paths,
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['labelmaker3d'])])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                subsampling=1,
                                pred_template='pred_consensus_5_scannet/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['consensus'])])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                subsampling=1,
                                pred_template='pred_consensus_noscannet/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['cons. w\o sn'])])

print('Semantic NeRF')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'id',
                                'wn199-merged-v2',
                                subsampling=2,
                                pred_template=semantic_nerf_paths,
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['SemanticNerf'])])

print('NoScannet')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                subsampling=2,
                                pred_template=noscannet_paths,
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['No Scannet'])])

display(df.T)

columns = ['tAcc', 'mIoU', 'mAcc']
# for i, x in enumerate(matcher.right_ids):
#     if confmat.sum(0)[i+1] > 0:
#         columns.append(classid2wn[x])
# df[columns].sort_values('mIoU', ascending=False).T


In [ ]:
# 2d evaluation performance scannet

scenes = [
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0000_00/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0458_00/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0164_02/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0518_00/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0474_01/',
]

overwrite_confmat = True
pd.set_option("display.float_format", "{:.3f}".format)
pd.set_option('display.max_rows', 500)

print('ovseg')
matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'wn199',
                                'wn199-merged-v2',
                                pred_template='pred_ovseg_wn_nodef/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                subsampling=1,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.DataFrame(d, index=['OVSeg']);

print('Intern')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'ade20k',
                                'wn199-merged-v2',
                                subsampling=1,
                                pred_template='pred_internimage/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['InternImage'])])


print('CMX')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'nyu40id',
                                'wn199-merged-v2',
                                subsampling=1,
                                pred_template='pred_cmx/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['CMX'])])

print('mask3d')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'id',
                                'wn199-merged-v2',
                                subsampling=1,
                                pred_template='pred_mask3d_rendered/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['Mask3D'])])


display(df.T)

columns = ['tAcc', 'mIoU', 'mAcc']
# for i, x in enumerate(matcher.right_ids):
#     if confmat.sum(0)[i+1] > 0:
#         columns.append(classid2wn[x])
# df[columns].sort_values('mIoU', ascending=False).T


In [ ]:
# 2d evaluation performance replica

scenes = [
          # f'{PATH_PREFIX}/scratch/scratch/scannetter/room_0_1/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/room_1_1/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/room_2_1/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/office_0_1/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/office_1_1/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/office_2_1/',
          # f'{PATH_PREFIX}/scratch/scratch/scannetter/office_3_1/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/office_4_1/',


]

sdfstudio_paths = [
                    'pred_sdfstudio_2023-08-03_210125/{k:05d}.png',
                    'pred_sdfstudio_2023-08-03_210350/{k:05d}.png',
                    'pred_sdfstudio_2023-08-03_201030/{k:05d}.png',
                    'pred_sdfstudio_2023-08-03_201254/{k:05d}.png',
                    'pred_sdfstudio_2023-08-03_201821/{k:05d}.png',
                    'pred_sdfstudio_2023-08-03_204934/{k:05d}.png'
                   
]

overwrite_confmat = True
pd.set_option("display.float_format", "{:.3f}".format)
pd.set_option('display.max_rows', 500)

print('ovseg')
matcher = LabelMatcher('wn199', 'replicaid')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'wn199',
                                'replicaid',
                                pred_template='pred_ovseg_wn_nodef/{k}.png',
                                label_template='semantic_class/semantic_class_{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                subsampling=1,
                                n_jobs=8)

d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.DataFrame(d, index=['OVSeg']);

print('Intern')
matcher = LabelMatcher('ade20k', 'replicaid')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'ade20k',
                                'replicaid',
                                subsampling=1,
                                pred_template='pred_internimage/{k}.png',
                                label_template='semantic_class/semantic_class_{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['InternImage'])])


print('CMX')
matcher = LabelMatcher('nyu40id', 'replicaid')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'nyu40id',
                                'replicaid',
                                subsampling=1,
                                pred_template='pred_cmx/{k}.png',
                                label_template='semantic_class/semantic_class_{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['CMX'])])

print('mask3d')
matcher = LabelMatcher('id', 'replicaid')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'id',
                                'replicaid',
                                subsampling=1,
                                pred_template='pred_mask3d_rendered/{k}.png',
                                label_template='semantic_class/semantic_class_{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['Mask3D'])])

print('consensus')
matcher = LabelMatcher('wn199', 'replicaid')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'wn199',
                                'replicaid',
                                subsampling=1,
                                pred_template='pred_wn_consensus/{k}.png',
                                label_template='semantic_class/semantic_class_{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['Conensus'])])

print('labelmaker3d')
matcher = LabelMatcher('wn199', 'replicaid')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'wn199',
                                'replicaid',
                                subsampling=1,
                                pred_template=sdfstudio_paths,
                                label_template='semantic_class/semantic_class_{k}.png',
                                overwrite_confmat=True,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['Labelmaker3D'])])

display(df.T)

columns = ['tAcc', 'mIoU', 'mAcc']
# for i, x in enumerate(matcher.right_ids):
#     if confmat.sum(0)[i+1] > 0:
#         columns.append(classid2wn[x])
# df[columns].sort_values('mIoU', ascending=False).T

In [ ]:
# consensus ablation

scenes = [
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0000_00/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0458_00/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0164_02/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0518_00/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0474_01/',
]

overwrite_confmat = True
pd.set_option("display.float_format", "{:.3f}".format)
pd.set_option('display.max_rows', 500)

method = 'cmx'
print(f'consensus w/o {method}')
matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                pred_template=f'pred_consensus_no_{method}' + '/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                subsampling=1,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.DataFrame(d, index=[f'consensus w/o {method}']);

method = 'intern'
print(f'consensus w/o {method}')
matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                pred_template=f'pred_consensus_no_{method}' + '/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                subsampling=1,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=[f'consensus w/o {method}'])])


method = 'mask3d'
print(f'consensus w/o {method}')
matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                pred_template=f'pred_consensus_no_{method}' + '/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                subsampling=1,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=[f'consensus w/o {method}'])])

method = 'ovseg'
print(f'consensus w/o {method}')
matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                pred_template=f'pred_consensus_no_{method}' + '/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                subsampling=1,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=[f'consensus w/o {method}'])])

metrics, confmat = segmentation_eval.evaluate_scenes(scenes,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                subsampling=1,
                                pred_template='pred_consensus_noscannet/{k}.png',
                                label_template='label_agile3d/{k}.png',
                                overwrite_confmat=overwrite_confmat,
                                n_jobs=8)
d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['consensus w\o scannet'])])



columns = ['tAcc', 'mIoU', 'mAcc']
# for i, x in enumerate(matcher.right_ids):
#     if confmat.sum(0)[i+1] > 0:
#         columns.append(classid2wn[x])
# df[columns].sort_values('mIoU', ascending=False).T

In [ ]:
df.T

# 3D eval

In [ ]:
PATH_PREFIX = '/home/weders/'

scenes = [f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0000_00/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0458_00/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0164_02/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0518_00/',
          f'{PATH_PREFIX}/scratch/scratch/scannetter/scene0474_01/',
]


labels_3d_labelmaker = [p.split('/')[-2] + '_labels_3d.txt' for p in sdfstudio_paths]
labels_3d_semantic_nerf = [p.split('/')[-2] + '_labels_3d.txt' for p in semantic_nerf_paths]
labels_3d_noscannet = [p.split('/')[-2] + '_labels_3d.txt' for p in noscannet_paths]


pd.set_option("display.float_format", "{:.3f}".format)

matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = pointcloud_eval.evaluate_scenes(scenes,
                                'id',
                                'wn199-merged-v2',
                                pred_dir='label-filt',
                                pred_pointcloud='{s}_vh_clean.ply',
                                pred_classes='label-filt_labels_3d.txt',
                                label_pointcloud='{s}_vh_clean.ply',
                                label_classes='label_agile3d_labels_3d.txt',
                                overwrite_confmat=True,
                                n_jobs=16)

d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.DataFrame(d, index=['scannet'])

matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = pointcloud_eval.evaluate_scenes(scenes,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                pred_dir='label-filt',
                                pred_pointcloud='{s}_vh_clean.ply',
                                pred_classes=labels_3d_labelmaker,
                                label_pointcloud='{s}_vh_clean.ply',
                                label_classes='label_agile3d_labels_3d.txt',
                                overwrite_confmat=True,
                                n_jobs=16)

d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['LabelMaker3D'])])

matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = pointcloud_eval.evaluate_scenes(scenes,
                                'id',
                                'wn199-merged-v2',
                                pred_dir='label-filt',
                                pred_pointcloud='{s}_vh_clean.ply',
                                pred_classes=labels_3d_semantic_nerf,
                                label_pointcloud='{s}_vh_clean.ply',
                                label_classes='label_agile3d_labels_3d.txt',
                                overwrite_confmat=True,
                                n_jobs=16)

d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['SemanticNerf'])])

matcher = LabelMatcher('id', 'wn199-merged-v2')
metrics, confmat = pointcloud_eval.evaluate_scenes(scenes,
                                'wn199-merged-v2',
                                'wn199-merged-v2',
                                pred_dir='label-filt',
                                pred_pointcloud='{s}_vh_clean.ply',
                                pred_classes=labels_3d_noscannet,
                                label_pointcloud='{s}_vh_clean.ply',
                                label_classes='label_agile3d_labels_3d.txt',
                                overwrite_confmat=True,
                                n_jobs=16)

d = {'tAcc': [metrics['tAcc']], 'mIoU': [metrics['mIoU']], 'mAcc': [metrics['mAcc']]}
d.update({classid2wn[x]: [metrics['iou'][i]] for i, x in enumerate(matcher.right_ids)})
df = pd.concat([df, pd.DataFrame(d, index=['LabelMaker3D w/o Scannet'])])


columns = ['tAcc', 'mIoU', 'mAcc']
for i, x in enumerate(matcher.right_ids):
    if confmat.sum(0)[i+1] > 0:
        columns.append(classid2wn[x])
df[columns].sort_values('mIoU', ascending=False).T